plots
- nc x h2            (obviously, the effect size drives power; also - PCOR MOSTest behaves as GCOR )
- dist x t           (most is always better than minp; minp power is growing when t increases; cauchy distribution => minp has thesame power as mostest)
- dist x T           (more traits = better power; and too many traits = problem for MultiABEL)
- link_int x rg_re   (less obviously, the effect size is best in the presence of phenotypic correlation AND in the absence of genetic correlation)
                     (also, it is not obvious - but known from before - that INT reduces power)
- comb x link_int    (type-I error analysis)
                     (INT is absolutely essential in the presence of non-linear effects)
                     (regularization is important - sometimes it improves power; in terms of improving correctness of the test, this needs better visualization - currently unclear)

ideas
- update "mostest_gen" script, excluding what is not need here
- show QQ plots on the same FacetGrids
- show how a&b&rcond depends on regularization? (can put labels on the QQ plots)
- draw causals on chr22, evaluate on all other chromosomes to have lower error bars. chr21 is enough to evaluate power, but for the null we may need a lot of observation.
- really nice to include MultiPheno
+ show that PCOR is the same as GCOR


findings (also described in the "plots" above)
- enabling INT may reduce the power (e.g comb=none & link=id) but it may improve it also (comb=none & link=exp)
- doing residualization does improve the power (e.g. comb=sum)

In [1]:

while True:
    pass

KeyboardInterrupt: 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



In [122]:
def filter(df, keep=[], excl=[], scope=[], verbose=False):
    mask_keep = df.index.notnull()
    for key, val in keep:
        if not isinstance(val, list): val=[val]
        mask_keep = mask_keep & (df[key].isin(val))
        if verbose: print('keep', key, val, sum(mask_keep))

    mask_excl = ~df.index.notnull()
    for key, val in excl:
        if not isinstance(val, list): val=[val]
        mask_excl = mask_excl | (df[key].isin(val))
        if verbose: print('excl', key, val, sum(mask_excl))

    mask_scope = df.index.notnull()
    for key, val in scope:
        if not isinstance(val, list): val=[val]
        mask_scope = mask_scope & (df[key].isin(val))
        if verbose: print('scope', key, val, sum(mask_scope))

    return df[~mask_scope | (mask_keep & ~mask_excl)].copy()

import json
def json_loads(fname):
    return json.loads(open(fname).read())

def qqplot(strat, *args, **kwargs):
    cm = plt.cm.get_cmap('tab10')
    mode = 'orig' if (strat != 2) else 'perm'
    use_strat = 0 if (strat==0) else 1
    colors = {'.MultiABEL.':cm.colors[3], '_GCORandEIG0.minp_':cm.colors[1], '_GCORandEIG0.most_':cm.colors[2], '_GCORandEIG275.most_':cm.colors[4], '_PCORandEIG275.most_':cm.colors[5]}
    linestyles = {'.MultiABEL.':'-', '_GCORandEIG0.minp_':'--', '_GCORandEIG0.most_':'-.', '_GCORandEIG275.most_':'-.', '_PCORandEIG275.most_':':'}

    handles = []
    for test in ['.MultiABEL.', '_GCORandEIG0.minp_', '_GCORandEIG0.most_', '_GCORandEIG275.most_', '_PCORandEIG275.most_']:
        
        fnames = []
        for basename in args[0].values:
            basename = results_folder + basename.split('/')[-1]
        
            fname = '{}_pheno.csv{}{}.sumstats.gz.png.json'.format(basename, test, mode)
            if not os.path.exists(fname): continue
            if fname in fnames: continue
            fnames.append(fname)
        
        for fname_index, fname in enumerate(fnames): #[:3]):
            j = json_loads(fname)

            plt.fill_between(j['x_ci'], j['lower_ci'], j['upper_ci'], color="0.8",label='_nolegend_');
            plt.plot(j['x_ci'], j['x_ci'], 'k--', label='_nolegend_')
            plt.plot(j['stratum'][use_strat]['x'], j['stratum'][use_strat]['y'], 
                     color = colors[test],
                     linestyle=('--' if (strat==1) else linestyles[test]), label=test)
            plt.plot(j['stratum'][use_strat]['x_dot'], j['stratum'][use_strat]['y_dot'], 
                     color = colors[test],
                     marker='.',  markersize=1, linewidth=0, alpha=0.5, label='_nolegend_')
            plt.xlim([0, 7.3])
            if strat==0: plt.ylim([0, 17])        
            else: plt.ylim([0, 7.3])
    
def qqplot_perm(*args, **kwargs):
    qqplot(2, *args, **kwargs)

def qqplot_null(*args, **kwargs):
    qqplot(1, *args, **kwargs)

def qqplot_orig(*args, **kwargs):
    qqplot(0, *args, *kwargs)
    
    
def render_figures_data(df, row, col, row_order, col_order, save_label, modes, hue_order_special=False, DO_QQPLOT=False, height=4):
    mode_filts = {'orig': [('chr', 'chr21'), ('mode', 'orig')],
                 'perm': [('chr', 'chr22'), ('mode', 'perm')],
                 'null': [('chr', 'chr22'), ('mode', 'orig')]}

    for mode in modes:
        data = filter(df, mode_filts[mode]).copy()
        data['test'] = [('_'.join([a, b]) if (a != b) else a) for a, b in zip(data['test'], data['test2'])]
        data.loc[data['test'] == 'true', 'test'] = 'expected'
        data.loc[data['test'] == 'minp_GCORandEIG0', 'test'] = 'minP'
        data.loc[data['test'] == 'most_GCORandEIG0', 'test'] = 'MOSTest'
        data.loc[data['test'] == 'most_PCORandEIG0', 'test'] = 'MOSTest_PCOR'
        data.loc[data['test'] == 'most_GCORandEIG275', 'test'] = 'MOSTest_reg'
        data.loc[data['test'] == 'most_PCORandEIG275', 'test'] = 'MOSTest_PCOR_reg'

        hue_order = (['expected'] if (mode!='orig') else ['']) + ['minP', 'MOSTest', 'abel'] + (['mqfam', 'mphen'] if (mode=='orig') else [])
        if hue_order_special:
            hue_order=(['expected'] if (mode!='orig') else ['']) + ['minP', 'MOSTest', 'MOSTest_PCOR', 'MOSTest_reg', 'MOSTest_PCOR_reg']+(['', ''] if (mode=='orig') else [])
        
        g = sns.catplot(x="thresh", y="frac_sig", hue="test", row=row, col=col, data=data, height=height, aspect=1.5, kind='bar', ci='sd', row_order=row_order, col_order = col_order,
                        hue_order=hue_order)
        g.set(xlabel='p-value threshold', ylabel=('power' if (mode == 'orig') else 'type I error'))
        if mode!='orig': g.set(yscale="log")
        if mode!='orig': g.set(ylim=[1e-8, 1])
        g.savefig("{}/{}_{}_{}.png".format(figures_folder, runid, save_label, mode))
        

        data_gb=data.groupby(['thresh', 'test', row, col]).agg({'frac_sig':['count']}).reset_index()
        print('max/min per group: ', data_gb['frac_sig'].max().values[0], data_gb['frac_sig'].min().values[0])
        #print(data['test'].unique())
        if DO_QQPLOT and (mode=='orig'):
            g = sns.FacetGrid(data, row=row, col=col, height=height, aspect=1.3,row_order=row_order, col_order=col_order)
            g.map(qqplot_null, 'basename')
            g.savefig("{}/{}_{}_{}.qq.null.png".format(figures_folder, runid, save_label, mode))

            g = sns.FacetGrid(data, row=row, col=col, height=height, aspect=1.3,row_order=row_order, col_order=col_order)
            g.map(qqplot_perm, 'basename')
            g.savefig("{}/{}_{}_{}.qq.perm.png".format(figures_folder, runid, save_label, mode))

            g = sns.FacetGrid(data, row=row, col=col, height=height, aspect=1.3,row_order=row_order, col_order=col_order)
            g.map(qqplot_orig, 'basename')
            g.savefig("{}/{}_{}_{}.qq.orig.png".format(figures_folder, runid, save_label, mode))    

In [ ]:
DO_QQPLOT = True

import os
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks")

#results_folder = '/home/oleksanf/vmshare/data/mostest_simu/run24_results/'; files = glob.glob(results_folder +'*table*'); figures_folder = '/home/oleksanf/vmshare/data/mostest_simu/run24_figures'; runid='24';
#results_folder = '/home/oleksanf/vmshare/data/mostest_simu/run23_results/'; files = glob.glob(results_folder +'*table*'); figures_folder = '/home/oleksanf/vmshare/data/mostest_simu/run23_figures'; runid='23';
#results_folder = '/home/oleksanf/vmshare/data/mostest_simu/run22_results/'; files = glob.glob(results_folder +'*table*'); figures_folder = '/home/oleksanf/vmshare/data/mostest_simu/run22_figures'; runid='22';
#results_folder = '/home/oleksanf/vmshare/data/mostest_simu/run21_results/'; files = glob.glob(results_folder + '*table*'); figures_folder = '/home/oleksanf/vmshare/data/mostest_simu/run21_figures'; runid='21';
results_folder = '/home/oleksanf/vmshare/data/mostest_simu/run30_results/'; files = glob.glob(results_folder +'*table*'); figures_folder = '/home/oleksanf/vmshare/data/mostest_simu/run30_figures'; runid='30';

# Main analysis across all chromosomes
#results_folder = '/home/oleksanf/vmshare/data/mostest_simu/run32/'; files = glob.glob(results_folder +'*table*'); figures_folder = '/home/oleksanf/vmshare/data/mostest_simu/run32_figures'; runid='32';

# Main-text simulation figure (35 features, "realistic" scenario)
#results_folder = '/home/oleksanf/vmshare/data/mostest_simu/run35_results/'; files = glob.glob(results_folder +'*table*'); figures_folder = '/home/oleksanf/vmshare/data/mostest_simu/run35_figures'; runid='35';

df=pd.concat([pd.read_csv(file, sep='\t') for file in files if (os.stat(file).st_size > 0)])
df.loc[df['test']=='abel', 'test2']='abel'


df_true = df.copy()
df_true['test'] = 'true'
df_true['test2'] = 'true'
df_true['valA'] = 0
df_true['valB'] = 0
df_true['valRCOND'] = 0
df_true['basename'] = ''
df_true['matfile'] = ''
df_true['num_sig'] = 1
df_true['num_val'] = np.divide(1, df_true['thresh'].values)
df_true['frac_sig'].values[:] = df_true['thresh'].values
df_true.drop_duplicates(inplace=True)

df = pd.concat([df, df_true])

df['rg_re'] = ['{}_{}'.format(x, y) for x, y in zip(df['rg'].values, df['re'].values)]
df['link_int'] = ['{}_{}'.format(x, y) for x, y in zip(df['link'].values, df['int'].values)]
df['test_test2'] = ['{}_{}'.format(x, y) for x, y in zip(df['test'].values, df['test2'].values)]
df['comb_test_test2'] = ['{}_{}_{}'.format(x, y, z) for x, y, z in zip(df['comb'], df['test'].values, df['test2'].values)]

df['COR'] = 'None'
df['COR'][df['test2'].str.contains('PCOR')] = 'PCOR'
df['COR'][df['test2'].str.contains('GCOR')] = 'GCOR'

df=df[df.thresh != 5e-7].copy()
df = filter(df, excl=[('test', 'true')], scope=[('mode', 'orig'), ('chr', 'chr21')])

In [41]:
df.to_csv(results_folder.rstrip('/')+'.csv.gz',index=False, sep='\t')

In [ ]:
# Main Figure 1
fig = plt.figure(figsize=(20,10))
for plot_index, plot_label in enumerate('A B C D E F'.split()):
    mode = ['perm', 'null', 'orig',  'perm', 'null', 'orig' ] [plot_index]
    
    mode_filt = [('chr', 'chr21' if (mode=='orig') else 'chr22') , ('mode', 'orig')] if (mode != 'perm') else [('mode', 'perm')]
    data = filter(df, keep=[('T', 35), ('h2', 0.004)]+mode_filt)
    data = filter(data, keep=[('test2', ['GCORandEIG0'])], scope=[('test', ['most', 'minp'])])
    data['test'] = [('_'.join([a, b]) if (a != b) else a) for a, b in zip(data['test'], data['test2'])]
    data['test'][data['test'] == 'true'] = 'expected'
    data['test'][data['test'] == 'minp_GCORandEIG0'] = 'minP'
    data['test'][data['test'] == 'most_GCORandEIG0'] = 'MOSTest'
    data['test'][data['test'] == 'most_PCORandEIG0'] = 'MOSTest_PCOR'
   
    ax=plt.subplot(2,3,plot_index+1)
    ax.text(-0.1, 1.05, plot_label, transform=ax.transAxes, fontsize=16, fontweight='bold', va='top', ha='right')
    if plot_index in [0,1,2]:
        g = sns.barplot(x="thresh", y="frac_sig", hue="test", data=data, ci='sd', #height=5, aspect=1.5, kind='bar', 
                        hue_order=(['expected'] if (mode!='orig') else ['expected']) + ['minP', 'MOSTest', 'abel'] + (['mqfam', 'mphen'] if (mode=='orig') else []))
        if mode!='orig': g.set(yscale="log")
        if mode!='orig': g.set(ylim=[1e-8, 1])
    if plot_index == 2:
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    if plot_index in [0, 1]:
        ax.get_legend().remove()

    if plot_index==3:
        qqplot_perm(data['basename'])
    if plot_index==4:
        qqplot_null(data['basename'])
    if plot_index==5:
        qqplot_orig(data['basename'])

    data_gb=data.groupby(['thresh', 'test', 't', 'h2']).agg({'frac_sig':['count']}).reset_index()
    print('max/min per group: ', data_gb['frac_sig'].max().values[0], data_gb['frac_sig'].min().values[0])
        
fig.savefig("{}/{}_mimic_real.png".format(figures_folder,  runid))



In [110]:
# Supplementary Figure 1 
data = filter(df, keep=[('T', 25), ('t', 25), ('dist', 'norm'), ('rg', 'eye'), ('re', 'eye'), ('comb', 'none'), ('link', 'id'), ('int', True)])
data = filter(data, keep=[('test2', ['GCORandEIG0', 'PCORandEIG0'])], scope=[('test', ['most', 'minp'])])
data = filter(data, excl=[('test2', ['PCORandEIG0'])], scope=[('test', ['minp'])])
render_figures_data(df=data, row='nc', col='h2', row_order = [1000, 100, 10], col_order = [0.0004, 0.004, 0.04], save_label="SFig1_nc_h2", modes=['orig', 'null']  )



In [ ]:
# Supplementary Figure 2
data = filter(df, keep=[('nc', 100),('T', [2,10,25,50,100]), ('h2', [0.04, 0.004, 0.0004]), ('dist', 'norm') ,('rg', 'eye'), ('re', 'eye'), ('link', 'id'),  ('int', True), ('comb', 'none')], verbose=False)
data = filter(data, keep=[('test2', 'GCORandEIG0')], scope=[('test', ['most', 'minp'])], verbose=False)
data = data[data['t'].values == data['T'].values].copy()
render_figures_data(df=data, row='T', col='h2', row_order = None, col_order = None, save_label="SFig2_h2_T", modes=['orig', 'null']  )


In [ ]:
# Supplementary Figure 7
data = filter(df, keep=[('nc', 100),('T', [25,50,100]), ('h2', [0.04, 0.004, 0.0004]), ('dist', 'norm') ,('rg', 'eye'), ('re', 'eye'), ('link', 'id'),  ('int', True), ('comb', 'none')], verbose=False)
data = filter(data, keep=[('test2', 'GCORandEIG0')], scope=[('test', ['most', 'minp'])], verbose=False)
data = data[data['t'].values == data['T'].values].copy()
render_figures_data(df=data, row='T', col='h2', row_order = None, col_order = None, save_label="SFig7_h2_T", modes=['orig', 'null', 'perm'] , DO_QQPLOT=True, height=3 )

In [59]:
# Supplementary Figure 3
data = filter(df, keep=[('dist', 'sparse'), ('T', [2,4,10,20,50,100]), ('t', [1,2,4,10]), ('nc', 100),  ('h2', 0.04),  ('rg', 'eye'), ('re', 'eye'), ('link', 'id'),  ('int', True), ('comb', 'none')], verbose=False)
data = filter(data, keep=[('test2', 'GCORandEIG0')], scope=[('test', ['most', 'minp'])])
render_figures_data(df=data, row='T', col='t', row_order = None, col_order = None, save_label="SFig3_sparse", modes=['orig', 'null']  )

In [ ]:
# Supplementary Figure 4
data = filter(df, excl=[('t', 25)], keep=[('T', 25), ('nc', 100),  ('h2', 0.04),  ('rg', 'eye'), ('re', 'eye'), ('link', 'id'),  ('int', True), ('comb', 'none')], verbose=False)
data = filter(data, keep=[('test2', 'GCORandEIG0')], scope=[('test', ['most', 'minp'])])
render_figures_data(df=data, row='t', col='dist', col_order = ['norm', 'cauchy', 'sparse'], row_order = None, save_label="SFig4_cauchy", modes=['orig', 'null']  )

In [ ]:
# Supplementary Figure 5
data = filter(df, keep=[('T', 25), ('t', 25), ('nc', 100),  ('h2', 0.004), ('dist', 'norm'), ('comb', 'none')])
data = filter(data, keep=[('test2', 'GCORandEIG0')], scope=[('test', ['most', 'minp'])])
#data = filter(data, excl=[('link', 'exp')], scope=[('int', False)])
render_figures_data(df=data, row='link_int', col='rg_re', row_order=['id_False', 'id_True', 'exp_False', 'exp_True'], col_order = ['eye_eye', 'eye_real', 'real_eye', 'real_real'], save_label="SFig5_link_int_rg_re", modes=['orig', 'null', 'perm'])

In [ ]:
# Supplementary Figure 6
data = filter(df, keep=[('T', 25), ('t', 25), ('nc', 100),  ('h2', 0.004), ('dist', 'norm'), ('comb', 'none')])
data = filter(df, keep=[('T', 25), ('t', 25), ('nc', 100),  ('h2', 0.004), ('dist', 'norm'), ('comb', ['sum']), ('rg', 'eye'), ('re', 'eye')])
#data = filter(df, keep=[('T', 10), ('t', 10), ('nc', 40),  ('h2', 0.004), ('dist', 'norm'),  ('rg', 'eye'), ('re', 'eye')] + mode_filt, verbose=False)
data = filter(data, excl=[('test2', ['_pheno.csv', 'PCORandEIG100', 'PCORandEIG20', 'PCORandEIG10'])])
data = filter(data, excl=[('test2', ['GCORandEIG100', 'GCORandEIG10', 'GCORandEIG20', 'minp_GCORandEIG275'])], scope=[('test', ['most', 'minp'])])
render_figures_data(df=data, row='comb', col='link_int', row_order=['sum'], col_order = ['exp_False', 'id_True', 'exp_True'], save_label="SFig6_sum", modes=['orig', 'null', 'perm'], hue_order_special=True, DO_QQPLOT=True)

In [ ]:
# Supplementary Figure 6
for mode in ['orig', 'perm', 'null'] :
    mode_filt = [('chr', 'chr21' if (mode=='orig') else 'chr22') , ('mode', 'orig')] if (mode != 'perm') else [('mode', 'perm')]
    data = filter(df, keep=[('T', 25), ('t', 25), ('nc', 100),  ('h2', 0.004), ('dist', 'norm'), ('comb', ['sum']), ('rg', 'eye'), ('re', 'eye')] + mode_filt)
    #data = filter(df, keep=[('T', 10), ('t', 10), ('nc', 40),  ('h2', 0.004), ('dist', 'norm'),  ('rg', 'eye'), ('re', 'eye')] + mode_filt, verbose=False)
    data = filter(data, excl=[('test2', ['_pheno.csv', 'PCORandEIG275', 'PCORandEIG100', 'PCORandEIG20', 'PCORandEIG10', 'PCORandEIG0'])])
    data = filter(data, excl=[('test2', ['GCORandEIG100', 'GCORandEIG10', 'GCORandEIG20', 'minp_GCORandEIG275'])], scope=[('test', ['most', 'minp'])])
    data['test'] = [('_'.join([a, b]) if (a != b) else a) for a, b in zip(data['test'], data['test2'])]
    #data = filter(data, keep=[('thresh', '0.01'), ('h2', '0.004'), ('nc', 100)])   # for debugging, to keep just one plot

    g = sns.catplot(x="thresh", y="frac_sig", hue="test", row="comb", col="link_int", data=data, height=5, aspect=1.5, kind='bar', ci='sd',
                    row_order=['sum'], col_order = ['exp_False', 'id_True', 'exp_True'],
                    hue_order=(['true'] if (mode!='orig') else ['']) + ['most_GCORandEIG0', 'minp_GCORandEIG0', '', 'most_GCORandEIG275']+(['', ''] if (mode=='orig') else []))
    if mode!='orig': g.set(yscale="log")
    if mode!='orig': g.set(ylim=[1e-8, 1])
    g.savefig("{}/{}_combSUM_link_int_{}.png".format(figures_folder, runid, mode))

    data_gb = data.groupby(['thresh', 'test', 'comb', 'link_int']).agg({'frac_sig':['count']}).reset_index()
    print('max/min per group: ', data_gb['frac_sig'].max().values[0], data_gb['frac_sig'].min().values[0])
    DO_QQPLOT=True
    if DO_QQPLOT and (mode=='orig'):
        g = sns.FacetGrid(data, row="comb", col="link_int", height=4, aspect=1.3,
                         row_order=['sum'], col_order = ['exp_False', 'id_True', 'exp_True'])
        g.map(qqplot_null, 'basename')
        g.savefig("{}/{}_combSUM_link_int_{}.qq.null.png".format(figures_folder, runid, mode))

        g = sns.FacetGrid(data, row="comb", col="link_int", height=4, aspect=1.3,
                          row_order=['sum'], col_order = ['exp_False', 'id_True', 'exp_True'])
        g.map(qqplot_orig, 'basename')
        g.savefig("{}/{}_combSUM_link_int_{}.qq.orig.png".format(figures_folder, runid, mode))    
        
        g = sns.FacetGrid(data, row="comb", col="link_int", height=4, aspect=1.3,
                          row_order=['sum'], col_order = ['exp_False', 'id_True', 'exp_True'])
        g.map(qqplot_perm, 'basename')
        g.savefig("{}/{}_combSUM_link_int_{}.qq.perm.png".format(figures_folder, runid, mode))            

In [ ]:
for mode in ['orig', 'perm', 'null'] :
    mode_filt = [('chr', 'chr21' if (mode=='orig') else 'chr22') , ('mode', 'orig')] if (mode != 'perm') else [('mode', 'perm')]
    data = filter(df, keep=[('T', 25), ('t', 25), ('nc', 100),  ('h2', 0.004), ('dist', 'norm'), ('comb', ['none']), ('rg', 'eye'), ('re', 'eye')] + mode_filt)
    #data = filter(df, keep=[('T', 10), ('t', 10), ('nc', 40),  ('h2', 0.004), ('dist', 'norm'),  ('rg', 'eye'), ('re', 'eye')] + mode_filt, verbose=False)
    data = filter(data, excl=[('test2', ['_pheno.csv', 'PCORandEIG275', 'PCORandEIG100', 'PCORandEIG20', 'PCORandEIG10', 'PCORandEIG0'])])
    data = filter(data, excl=[('test2', ['GCORandEIG100', 'GCORandEIG10', 'GCORandEIG20', 'minp_GCORandEIG275'])], scope=[('test', ['most', 'minp'])])
    data['test'] = [('_'.join([a, b]) if (a != b) else a) for a, b in zip(data['test'], data['test2'])]
    #data = filter(data, keep=[('thresh', '0.01'), ('h2', '0.004'), ('nc', 100)])   # for debugging, to keep just one plot

    g = sns.catplot(x="thresh", y="frac_sig", hue="test", row="comb", col="link_int", data=data, height=5, aspect=1.5, kind='bar', ci='sd',
                    row_order=['none'], col_order = ['id_False', 'exp_False', 'id_True', 'exp_True'],
                    hue_order=(['true'] if (mode!='orig') else ['']) + ['most_GCORandEIG0', 'minp_GCORandEIG0', 'abel', 'most_GCORandEIG275']+(['mqfam', 'mphen'] if (mode=='orig') else []))
    if mode!='orig': g.set(yscale="log")
    if mode!='orig': g.set(ylim=[1e-8, 1])
    g.savefig("{}/{}_combNONE_link_int_{}.png".format(figures_folder, runid, mode))

    data_gb = data.groupby(['thresh', 'test', 'comb', 'link_int']).agg({'frac_sig':['count']}).reset_index()
    print('max/min per group: ', data_gb['frac_sig'].max().values[0], data_gb['frac_sig'].min().values[0])

    if DO_QQPLOT and (mode=='orig'):
        g = sns.FacetGrid(data, row="comb", col="link_int", height=4, aspect=1.3,
                         row_order=['none'], col_order = ['id_False', 'exp_False', 'id_True', 'exp_True'])
        g.map(qqplot_null, 'basename')
        g.savefig("{}/{}_combNONE_link_int_{}.qq.null.png".format(figures_folder, runid, mode))

        g = sns.FacetGrid(data, row="comb", col="link_int", height=4, aspect=1.3,
                          row_order=['none'], col_order = ['id_False', 'exp_False', 'id_True', 'exp_True'])
        g.map(qqplot_orig, 'basename')
        g.savefig("{}/{}_combNONE_link_int_{}.qq.orig.png".format(figures_folder, runid, mode))    
        
        g = sns.FacetGrid(data, row="comb", col="link_int", height=4, aspect=1.3,
                          row_order=['none'], col_order = ['id_False', 'exp_False', 'id_True', 'exp_True'])
        g.map(qqplot_perm, 'basename')
        g.savefig("{}/{}_combNONE_link_int_{}.qq.perm.png".format(figures_folder, runid, mode))            

In [ ]:
for mode in ['orig', 'perm', 'null'] :
    mode_filt = [('chr', 'chr21' if (mode=='orig') else 'chr22') , ('mode', 'orig')] if (mode != 'perm') else [('mode', 'perm')]
    data = filter(df, keep=[('T', 25), ('t', 25), ('nc', 100), ('link', 'exp'), ('int', True), ('h2', 0.004), ('dist', 'norm'), ('comb', 'none')] + mode_filt, verbose=False)
    data = filter(data, excl=[('COR', 'PCOR')])
    data = filter(data, keep=[('test2', 'GCORandEIG0')], scope=[('test', 'minp')])
    data = filter(data, keep=[('test', ['most', 'true', 'minp'])])
    
    data['test'] = [('_'.join([a, b]) if (a != b) else a) for a, b in zip(data['test'], data['test2'])]
    #data = filter(data, keep=[('thresh', '0.01')])   # for debugging, to keep just one plot

    hue_order=(['true'] if (mode!='orig') else ['']) + ['minp_GCORandEIG0'] + ['most_GCORandEIG{}'.format(x) for x in [0,5,10,15,20,24]]
    g = sns.catplot(x="thresh", y="frac_sig", hue="test", row="h2", col="rg_re", data=data, height=5, aspect=1.5, kind='bar', ci='sd',
                    col_order = ['eye_eye', 'eye_real', 'real_eye', 'real_real'], hue_order =  hue_order)
    if mode!='orig': g.set(yscale="log")
    if mode!='orig': g.set(ylim=[1e-8, 1])
    g.savefig("{}/{}_regularize_{}.png".format(figures_folder, runid, mode))
    
    data_gb = data.groupby(['thresh', 'test', 'test2', 'rg_re']).agg({'frac_sig':['count']}).reset_index()
    print('max/min per group: ', data_gb['frac_sig'].max().values[0], data_gb['frac_sig'].min().values[0])

    if DO_QQPLOT and (mode=='orig'):
        g = sns.FacetGrid(data,  row="link_int", col="rg_re", height=4, aspect=1.3,
                         col_order = ['eye_eye', 'eye_real', 'real_eye', 'real_real'],hue_order =  hue_order)
        g.map(qqplot_null, 'basename')
        g.savefig("{}/{}_regularize_{}.qq.null.png".format(figures_folder, runid, mode))

        g = sns.FacetGrid(data,  row="link_int", col="rg_re", height=4, aspect=1.3,
                           col_order = ['eye_eye', 'eye_real', 'real_eye', 'real_real'], hue_order =  hue_order)
        g.map(qqplot_orig, 'basename')
        g.savefig("{}/{}_regularize_{}.qq.orig.png".format(figures_folder, runid, mode))    
        
        g = sns.FacetGrid(data,  row="link_int", col="rg_re", height=4, aspect=1.3,
                           col_order = ['eye_eye', 'eye_real', 'real_eye', 'real_real'],hue_order =  hue_order)
        g.map(qqplot_perm, 'basename')
        g.savefig("{}/{}_regularize_{}.qq.perm.png".format(figures_folder, runid, mode))            

In [ ]:

DO_QQPLOT=True
import os

for mode in ['orig', 'perm', 'null'] :
    mode_filt = [('chr', 'chr21' if (mode=='orig') else 'chr22') , ('mode', 'orig')] if (mode != 'perm') else [('mode', 'perm')]
    data = filter(df, keep=[('nc', 100), ('T', 25), ('h2', [0.004]), ('dist', 'norm') ,('rg', 'eye'), ('re', 'eye'), ('link', 'id'),  ('int', True), ('comb', 'none')] + mode_filt, verbose=False)
    data = filter(data, keep=[('test2', 'GCORandEIG0')], scope=[('test', ['most', 'minp'])], verbose=False)
    data = data[data['t'].values == data['T'].values].copy()
        
    data['test'] = [('_'.join([a, b]) if (a != b) else a) for a, b in zip(data['test'], data['test2'])]
    #data = filter(data, keep=[('thresh', '0.01')])   # for debugging, to keep just one plot

    g = sns.catplot(x="thresh", y="frac_sig", hue="test", row="h2", col="T", data=data, height=4, aspect=1.3, kind='bar', ci='sd',
                    hue_order=(['true'] if (mode!='orig') else ['']) + [ 'most_GCORandEIG0', 'minp_GCORandEIG0', 'abel']+(['mqfam', 'mphen'] if (mode=='orig') else []))
    if mode!='orig': g.set(yscale="log")
    if mode!='orig': g.set(ylim=[1e-8, 1])
    #g.savefig("{}/{}_dist_T_{}.png".format(figures_folder, runid, mode))

    data_gb = data.groupby(['thresh', 'test', 'h2', 'T']).agg({'frac_sig':['count']}).reset_index()
    print('max/min per group: ', data_gb['frac_sig'].max().values[0], data_gb['frac_sig'].min().values[0])

    if DO_QQPLOT and (mode=='orig'):
        g = sns.FacetGrid(data, row="h2", col="T", height=4, aspect=1.3)
        g.map(qqplot_null, 'basename')
        #g.savefig("{}/{}_dist_T_{}.qq.null.png".format(figures_folder,  runid, mode))

        g = sns.FacetGrid(data, row="h2", col="T", height=4, aspect=1.3)
        g.map(qqplot_orig, 'basename')

        g = sns.FacetGrid(data, row="h2", col="T", height=4, aspect=1.3)
        g.map(qqplot_perm, 'basename')

        #g.savefig("{}/{}_dist_T_{}.qq.orig.png".format(figures_folder,  runid, mode))

In [191]:
cols = 'T t nc dist rg re h2 comb link int'.split()
df[cols].drop_duplicates().sort_values(cols).to_excel("{}/{}_scenarios.xlsx".format(figures_folder,  runid),index=False)



In [ ]:
/home/oleksanf/vmshare/data/dennis_MOST/2019_07_02_lambda1p0_maf0p005_all_most.outtag=run1.test.json'
2019_07_02_lambda1p0_maf0p005_aseg_vol_most.outtag=run1.test.json
2019_07_02_lambda1p0_maf0p005_dk_area_most.outtag=run1.test.json'
2019_07_02_lambda1p0_maf0p005_dk_thick_most.outtag=run1.test.json


In [ ]:
import json
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
fnames='all_most dk_area_most dk_thick_most aseg_vol_most all_minp dk_area_minp dk_thick_minp aseg_vol_minp'.split()
for idx, fname in enumerate(fnames):
    plt.subplot(2, 4, idx+1)
    j=json.loads(open('/home/oleksanf/vmshare/data/dennis_MOST/2019_07_02_lambda1p0_maf0p005_{}.outtag=run1.test.json'.format(fname)).read())
    data=j['univariate'][0]['qq_plot_data']
    plt.plot(data['data_logpvec'], data['hv_logp'])
    plt.plot(data['model_logpvec'], data['hv_logp'])
    plt.plot(data['hv_logp'], data['hv_logp'], 'k--')
    plt.title(fname)
    plt.xlim([0, 7.3]);
    plt.ylim([0, 7.3]);


In [ ]:
print(j['univariate'][0]['qq_plot_bins_data'][0]['qq_options']['title'])
print(j['univariate'][0]['qq_plot_bins_data'][4]['qq_options']['title'])
print(j['univariate'][0]['qq_plot_bins_data'][8]['qq_options']['title'])

In [ ]:
j['univariate'][0]['qq_plot_bins_data'][8].keys()

In [ ]:

import numpy as np
from numpy import ma
from scipy.interpolate import interp1d

def make_qq_plot(qq, ci=True):
    hv_logp = np.array(qq['hv_logp']).astype(float)
    data_logpvec = np.array(qq['data_logpvec']).astype(float)
    model_logpvec = np.array(qq['model_logpvec']).astype(float)
    ylim_data = max(hv_logp[np.isfinite(data_logpvec)])
    model_logpvec[hv_logp > ylim_data]=np.nan
    if ci: 
        q = 10**-data_logpvec; dq= 1.96*np.sqrt(q*(1-q)/qq['qq_options']['sum_data_weights']);
        y1=hv_logp
        x1=ma.filled(-ma.log10(q+dq), np.nan)  #left CI bound
        x2=ma.filled(-ma.log10(q-dq), np.nan)  #right CI bound
        if True:
            y2 = np.empty(hv_logp.shape); y2[:]=np.nan;
            y2[x2<np.nanmax(x1)]=interp1d(x1, y1)(x2[x2<np.nanmax(x1)])                #upper CI bound
            y2[np.isnan(y2)]=ylim_data  
            plt.fill_between(x2, y1, y2, color=(0.1843, 0.3098, 0.3098), alpha=0.25)
        else:
            plt.plot(x1,hv_logp,x2,hv_logp)
    hData = plt.plot(data_logpvec, hv_logp)
    hModel = plt.plot(model_logpvec, hv_logp)
    hNull = plt.plot(hv_logp, hv_logp, 'k--')


In [ ]:
import os
#fnames='all_most aseg_vol_most dk_area_most dk_thick_most'.split()
fnames='all_minp aseg_vol_minp dk_area_minp dk_thick_minp'.split()
fig=plt.figure(figsize=(12, 12))
for jdx, fname in enumerate(fnames):
    j=json.loads(open('/home/oleksanf/vmshare/data/dennis_MOST/2019_07_02_lambda1p0_maf0p005_{}.outtag=run1.test.json'.format(fname)).read())
    for iidx, idx in enumerate([2,5,8]):
    #for iidx, idx in enumerate([0,4,8]):
        qq =j['univariate'][0]['qq_plot_bins_data'][idx]
        plt.subplot(4,3,1+jdx*3+iidx)
        #plt.plot(qq['data_logpvec'], qq['hv_logp'])
        #plt.plot(qq['model_logpvec'], qq['hv_logp'])
        #plt.plot(qq['hv_logp'], qq['hv_logp'], 'k--')    
        make_qq_plot(qq)
        
        #a=qq['qq_options']['title'].replace('$', '').split(' ')[3].strip('[)').split(',')[0]
        #b=qq['qq_options']['title'].replace('$', '').split(' ')[-2].strip('[)').replace('Infinity', 'inf').replace(',','-')
        #plt.title('MAF>{}, LD={}'.format(a,b))
        #plt.title(qq['qq_options']['title'].replace('$', ''))
        
        if jdx==0: plt.title(['TLD<40', '40<TLD<106', 'TLD>106'][iidx])
        if iidx==0: plt.ylabel(['All', 'Subcortical', 'Area', 'Thickness'][jdx],fontsize=12)
                              
        
        plt.xlim([0, 7.3]);
        plt.ylim([0, 7.3]);   

fig.subplots_adjust(hspace=0.35);
fig.text(0.5, 0.075, 'Expected(-$log_{10}$ p-value)', ha='center',fontsize=16);
fig.text(0.050, 0.5, 'Observed(-$log_{10}$ p-value)', va='center', rotation='vertical',fontsize=16);
plt.savefig('/home/oleksanf/vmshare/data/dennis_MOST/MOSTest.minp.TLD.png', bbox_inches='tight')
plt.savefig('/home/oleksanf/vmshare/data/dennis_MOST/MOSTest.minp.TLD.svg', bbox_inches='tight')

In [ ]:
import glob
from dateutil import parser


#fname = '/home/oleksanf/vmshare/data/mostest_simu/run32/simuV3_T=50_t=50_nc=100_dist=norm_rg=eye_re=eye_h2=0.04_comb=none_link=id_int=True_rep=9.log'

fnames = [x for x in glob.glob('/home/oleksanf/vmshare/data/mostest_simu/run32/simuV3_*log') if (('_causals_bfile' not in x) and ('_mvplink_' not in x))]
def is_time(line):
    if '=command elapsed time:' in line:
        interval = parser.parse(line.strip().split(': ')[-1],fuzzy=True).time()
        return interval.second +60*interval.minute + 3600*interval.hour
    return -1

all_data = []
for fname in fnames:
    #print(fname)
    is_command=-1
    d=dict([tuple(x.split('=', 2)) for x in fname.split('/')[-1].split('_', 1)[-1].replace('.log', '').split('_')])
    
    for line in open(fname).readlines():
        if "command start time" in line: is_command = 0; continue
        if is_command >=0: is_command += 1
        if (is_command >= 7) or ("command end time" in line) or ("Beginning analysis at" in line):
            is_command = -1
            command_type = ''

        if ('matlab -nodisplay -nosplash' in line ) and ('zmat_name' in line):    command_type='time_mostest_rerun'
        if ('matlab -nodisplay -nosplash' in line ) and ('zmat_name' not in line): command_type='time_mostest_run'
        if ('require(MultiABEL)' in line ): command_type='time_mabel'
        if ('library(MultiPhen)' in line ): command_type='time_mphen'
        if ('plink.multivariate' in line ): command_type='time_mqfam'

        secs = is_time(line)

        if (secs >= 0 ) and (command_type != ''):
            #print(command_type, secs)
            if command_type not in d: d[command_type] = []
            d[command_type].append(secs/60)  # in minutes

    for k in d:
        if isinstance(d[k], list):
            d[k] = np.median(d[k])
    all_data.append(d)
    
import pandas as pd
df=pd.DataFrame(all_data)
dfg=df.groupby('T comb dist h2 int link nc re rg t'.split()).median().reset_index()

for c in dfg.columns:
    if c.startswith('time_'):
        dfg[c] = np.floor(dfg[c].values*10)/10
        
dfg.columns = [x.replace('time_', '') for x in dfg.columns]
dfg.rename(columns={'mostest_run':'gwas', 'mostest_rerun':'MOSTest', 'mabel':'abel'},inplace=True)

for c in 'T t nc'.split():
    dfg[c]=dfg[c].astype(int)
dfg['h2']=dfg['h2'].astype(float)

cols='T t nc dist rg re h2 comb link int gwas MOSTest abel mqfam mphen'.split()
dfg[cols].sort_values(by=cols).to_excel('/home/oleksanf/vmshare/projects/2020_01_02_MOSTest_natcoms/perf.xlsx', index=False)